In [10]:
# Import Libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [11]:
# Read Dataframes
df_emissions   = pd.read_csv("co2-emissions-borough.csv")
df_consumption = pd.read_csv("energy-consumption-borough.csv")
df_population  = pd.read_csv("housing-density-borough.csv")

In [12]:
# Data Preparation for Emissions dataset
df_emissions_new = df_emissions[~df_emissions["Borough"].isin(["London","Unapportioned"]) & 
                                ~df_emissions["Sector"].isin(['Total']) &
                                ~df_emissions["Fuel"].isin(['Total'])][['Borough', 'LEGGI_Year', 'Sector', 'Fuel', 'KtCO2e']]


In [13]:
# Data Preparation for Consumption dataset
df_consumption_new = df_consumption[~df_consumption["Area"].isin(["London","Unapportioned"]) &
                                    ~df_consumption["Sector"].isin(['Total']) &
                                    ~df_consumption["Fuel"].isin(['Total'])][['Area', 'LEGGI_Year', 'Sector', 'Fuel', 'kWh']]


In [14]:
# Data Preparation for Population dataset
df_population = df_population[['Name', 'Year','Population_per_square_kilometre']]

In [15]:
# Merging datasets

# Merge emission and consumption datasets
dataframe_merged = df_emissions_new.merge(df_consumption_new,
                                          left_on = ['Borough', 'LEGGI_Year', 'Sector', 'Fuel'],
                                          right_on = ['Area', 'LEGGI_Year', 'Sector', 'Fuel'])

# Merge Population data
data_viz_df = dataframe_merged.merge(df_population,
                                     left_on= ['Borough', 'LEGGI_Year'],
                                     right_on = ['Name', 'Year'])

# select features
selected_features = ['Borough', 'Year','Sector', 
                     'Fuel', 'KtCO2e', 'kWh',  
                     'Population_per_square_kilometre']

# Final df
data_viz_df = data_viz_df[selected_features]

# Change data types
data_viz_df["KtCO2e"]  = data_viz_df["KtCO2e"].astype(float)
data_viz_df["kWh"]     = data_viz_df["kWh"].astype(float)

# Data manipulation for plotting
bar_chart_df = data_viz_df.groupby("Borough")["KtCO2e"].sum().reset_index()
ts_chart_df  = data_viz_df.groupby("Year")["KtCO2e"].sum().reset_index()
scatter_chart_df  = data_viz_df.groupby(["Borough","Year","Sector"]).agg({"KtCO2e":sum,"kWh":sum,
                                                                         "Population_per_square_kilometre": 'mean'}).reset_index()



In [16]:
# Subplots specifications
fig = make_subplots(rows = 2,
                    cols = 2,
                    row_heights=[0.55, 0.45],
                    column_widths=[0.4, 0.6],
                    specs=[[{"type": "bar"}, {"type": "scatter"}],
                           [{"type":"bar", "colspan": 2},None]],
                    subplot_titles=("Emission over Years", 
                                    "Scatter plot - Emissions vs consumption ", 
                                    "Emission by Borough"))


# Plot Emission by Borough
fig.add_trace(go.Bar(y = bar_chart_df.sort_values(by = "KtCO2e",ascending = False)["KtCO2e"],
                     x = bar_chart_df.sort_values(by = "KtCO2e",ascending = False)["Borough"],
                    showlegend=False,marker_color='rgb(158,202,225)',
                    marker_line_color='rgb(8,48,107)',
                    marker_line_width=1.5, opacity=0.6, name = "KtCo2"), 
             row=2, col=1)

# Plot Emission over Years
fig.add_trace(go.Bar(y = ts_chart_df["KtCO2e"],x = ts_chart_df["Year"],showlegend=False,
                    marker_color='rgb(158,202,225)',
                    marker_line_color='rgb(8,48,107)',
                    marker_line_width=1.5, opacity=0.6, name = "KtCo2"),
             row=1, col=1)


# Scatter plot - Emissions vs consumption - Industrial
fig.add_trace(go.Scatter(x = scatter_chart_df[scatter_chart_df["Sector"] == "Industrial and Commercial"]["KtCO2e"],
                         y = scatter_chart_df[scatter_chart_df["Sector"] == "Industrial and Commercial"]["kWh"],
                         marker=dict(size = scatter_chart_df[scatter_chart_df["Sector"] == "Industrial and Commercial"]["Population_per_square_kilometre"]/700),   
                         name = "Industrial",hovertemplate = 'KtCO2e: %{x}<extra></extra>   kWh: %{y}<extra></extra>',
                         mode = "markers"),
             row=1, col=2)

# Scatter plot - Emissions vs consumption - Domestic
fig.add_trace(go.Scatter(x = scatter_chart_df[scatter_chart_df["Sector"] == "Domestic"]["KtCO2e"],
                         y = scatter_chart_df[scatter_chart_df["Sector"] == "Domestic"]["kWh"],
                         marker=dict(size = scatter_chart_df[scatter_chart_df["Sector"] == "Domestic"]["Population_per_square_kilometre"]/700),   
                         name = "Domestic",hovertemplate = 'KtCO2e: %{x}<extra></extra>   kWh: %{y}<extra></extra>',
                         mode = "markers"),
             row=1, col=2)

# Scatter plot - Emissions vs consumption - Transport
fig.add_trace(go.Scatter(x = scatter_chart_df[scatter_chart_df["Sector"] == "Transport"]["KtCO2e"],
                         y = scatter_chart_df[scatter_chart_df["Sector"] == "Transport"]["kWh"],
                         marker=dict(size = scatter_chart_df[scatter_chart_df["Sector"] == "Transport"]["Population_per_square_kilometre"]/700),   
                         name = "Transport",hovertemplate = 'KtCO2e: %{x}<extra></extra>   kWh: %{y}<extra></extra>',
                         mode = "markers"),
             row=1, col=2)


# Figure layout
fig.update_layout(
    title = "Co2 Emissions in London",
    template="seaborn",
    width  = 1000,
    height = 700,
    margin = dict(r= 10, t=70, b=10, l=10),
    legend_title="Sector",hovermode='x unified'
)

# Figure axis plot 1
fig.update_xaxes(title_text="Year", row=1, col=1,
                 tickmode = 'linear')
fig.update_yaxes(title_text="KTCo2", row=1, col=1)

# Figure axis plot 2
fig.update_xaxes(title_text="KtCO2", row=1, col=2)
fig.update_yaxes(title_text="kWh", row=1, col=2)

# Figure axis plot 3
fig.update_xaxes(title_text="Borough", row=2, col=1)
fig.update_yaxes(title_text="KtCO2", row=2, col=1)


fig.show()